F2スコアが最も高くなる閾値を決定する

In [1]:
def make_only_threthold(sample_num = 100000):
    start = time.time()
    model.eval()
    steps = len(ds_valid)
    propotion_arr = list()
    labels_arr = list()

    # ラベル確率を推論
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader_valid,1):
            images = images.to(device)
            labels = labels.cpu().detach().numpy()
            labels_arr.extend(labels)
            outputs = torch.sigmoid(model(images))
            outputs = outputs.cpu().detach().numpy()
            propotion_arr.extend(outputs)
        #         outputs_topN = np.argsort(outputs, axis=1)[:,-extract_attribute:]
        #         for attr in outputs_topN:
        #             pred_attr.append(attr)
            if i % 10 == 0:
                elapsed_time = time.time() - start
                print('\r[%d/%d], time: %d分%d秒' % (min((i * batch_size),steps), steps, elapsed_time//60, int(elapsed_time % 60)))
                clear_output(wait=True)


    propotion_arr = np.asarray(propotion_arr)
    labels_arr = np.asarray(labels_arr)

    pc = deepcopy(propotion_arr)
    lc = deepcopy(labels_arr)
    pc = np.reshape(pc,-1)
    lc = np.reshape(lc,-1)
    idx = np.argsort(pc)
    pc = pc[idx]
    lc = lc[idx]

    TP = np.sum(labels_arr==1, axis=1)
    FN = np.zeros_like(TP)
    FP = np.sum(labels_arr==0, axis=1)
    TN = np.zeros_like(TP)

    f2 = np.zeros_like(TP)

    tmp_max = 0
    max_thr = 0
    pos = 0
    for i, thr in enumerate(np.logspace(-4,0,sample_num)):
        if i % 100 == 0:
            elapsed_time = time.time() - start
            print('\r[%d/%d], time: %d分%d秒' % (i, sample_num, elapsed_time//60, int(elapsed_time % 60)))
            clear_output(wait=True)
        while pos < len(pc) and pc[pos] < thr:
            if lc[pos] == 0:
                FP[idx[pos] // num_classes] -= 1
                TN[idx[pos] // num_classes] += 1
            else:
                TP[idx[pos] // num_classes] -= 1
                FN[idx[pos] // num_classes] += 1
#             if pos % 100000 == 0: print(pos)
            pos += 1

        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f2_arr = 5*(precision * recall) / (4*precision + recall)
        f2_arr[np.isnan(f2_arr)] = 0
        f2 = np.mean(f2_arr)
        if f2 > tmp_max:
            tmp_max = f2
            max_thr = thr
    return max_thr, tmp_max, 

上から$n$個の属性とその頻度を返す

#上位何個取るか
take_num = 1103

cnt_attribute = Counter()
for i in train_df.attribute_ids:
    cnt_attribute.update(i)

freq_attr = np.asarray(cnt_attribute.most_common(take_num))